# 第6章 Python 中的设计模式与范式

## 6.1 函数式编程：利用 Python 特性增强抽象


Python 并非纯粹的函数式语言，但它提供了强大的函数式编程工具，可以帮助我们编写更简洁、更少副作用的代码。

函数式编程（Functional Programming）是一种编程范式，它将计算视为数学函数的求值，并避免使用可变状态和数据修改。与命令式编程关注如何做不同，函数式编程更关注做什么。Python作为多范式语言，很好地融合了函数式编程的特性。

### 6.1.1 函数式编程的核心思想

在Python中实践函数式编程，主要体现以下几个核心思想：

1. 函数是一等公民

函数可以像普通数据一样被传递、赋值和返回。这使得高阶函数（接受函数作为参数或返回函数的函数）成为可能，为代码复用和抽象提供了强大工具。



In [1]:
# 函数可以赋值给变量
def greet(name):
    return f"你好, {name}!"


say_hello = greet
print(say_hello("小白"))  # 输出: 你好, 小白!


# 函数作为参数传递
def apply_twice(func, value):
    """将函数应用两次"""
    return func(func(value))


def add_one(x):
    return x + 1


print(apply_twice(add_one, 10))  # 输出: 12 (10+1+1)

你好, 小白!
12


2. 纯函数与不可变性

纯函数是指给定相同输入总是返回相同输出，且没有副作用（Side Effect）的函数。其中，确定性是指对于相同的输入，总是返回相同的输出；无副作用是指函数不会修改外部状态，也不依赖外部状态。

在实际编程中，我们尽量使用纯函数来减少程序的不确定性。

In [2]:
# 纯函数示例：无副作用，结果可预测
def calculate_circle_area(radius):
    return 3.14159 * radius * radius


# 非纯函数示例：依赖外部状态
count = 0


def increment():
    global count
    count += 1  # 有副作用
    return count


print(calculate_circle_area(10))  # 总是返回314.159
print(increment())  # 第一次调用返回1
print(increment())  # 第二次调用返回2，结果依赖调用次数

314.159
1
2



3. 声明式而非命令式

函数式编程鼓励使用声明式风格编写代码，即描述做什么而不是如何做。

In [3]:
# 命令式：描述详细步骤
numbers = [1, 2, 3, 4, 5]
result = []
for num in numbers:
    if num % 2 == 0:
        result.append(num * 2)

# 声明式：直接描述需求
result_fp = [num * 2 for num in numbers if num % 2 == 0]

print(result)  # 输出: [4, 8]
print(result_fp)  # 输出: [4, 8]

[4, 8]
[4, 8]


### 6.1.2 高阶函数：map、filter与推导式


高阶函数是函数式编程的核心工具。Python内置了\mintinline{python}{map}、\mintinline{python}{filter}等函数，同时提供了推导式语法，为函数式编程提供了多种表达方式。

1. 基础应用


In [4]:
# 使用高阶函数处理数据
numbers = [1, 2, 3, 4, 5]

# map：将函数应用于每个元素
squared = list(map(lambda x: x**2, numbers))
print(squared)  # 输出: [1, 4, 9, 16, 25]

# filter：过滤符合条件的元素
evens = list(filter(lambda x: x % 2 == 0, numbers))
print(evens)  # 输出: [2, 4]

# 列表推导式：更Pythonic的表达方式
squared_comprehension = [x**2 for x in numbers]
evens_comprehension = [x for x in numbers if x % 2 == 0]

print(squared_comprehension)  # 输出: [1, 4, 9, 16, 25]
print(evens_comprehension)  # 输出: [2, 4]

[1, 4, 9, 16, 25]
[2, 4]
[1, 4, 9, 16, 25]
[2, 4]



2. 惰性求值优势

map和filter返回的是迭代器，支持惰性求值，这在处理大数据时可以显著节省内存。


In [ ]:
# 生成大量数据
numbers = range(1_000_000)  # 不实际占用内存

# 列表推导式：立即计算并存储所有结果
# squared_list = [x**2 for x in numbers]  # 会占用大量内存

# map：只在需要时计算
squared_map = map(lambda x: x**2, numbers)

# 只取前3个结果进行演示，因为不需要存储所有结果，速度非常快
for i, value in enumerate(squared_map):
    if i >= 3:
        break
    print(value)  # 输出: 1, 4, 9

0
1
4



3.推导式的高级用法

推导式不仅适用于列表，也可用于字典、集合，并能处理更复杂的数据结构。


In [7]:
# 字典推导式：创建键值映射
fruits = ["apple", "banana", "cherry"]
fruit_lengths = {fruit: len(fruit) for fruit in fruits}
print(fruit_lengths)  # 输出: {'apple': 5, 'banana': 6, 'cherry': 6}

# 集合推导式：自动去重
numbers = [1, 2, 2, 3, 3, 3]
unique_squares = {x**2 for x in numbers}
print(unique_squares)  # 输出: {1, 4, 9}

# 嵌套推导式：处理多维数据
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flattened = [num for row in matrix for num in row]
print(flattened)  # 输出: [1, 2, 3, 4, 5, 6, 7, 8, 9]

{'apple': 5, 'banana': 6, 'cherry': 6}
{1, 4, 9}
[1, 2, 3, 4, 5, 6, 7, 8, 9]


### 6.1.3 偏函数：固化参数简化调用

In [8]:
def process_fruit(name, action, peeled=True, sliced=False):
    """处理水果"""
    result = f"{name}"
    if peeled:
        result += "（去皮）"
    if sliced:
        result += "（切片）"
    result += f" 进行{action}处理"
    return result

In [9]:
print(process_fruit("苹果", "榨汁", peeled=True, sliced=True))
print(process_fruit("香蕉", "制作沙拉", peeled=True, sliced=False))
print(process_fruit("橙子", "榨汁", peeled=True, sliced=True))

苹果（去皮）（切片） 进行榨汁处理
香蕉（去皮） 进行制作沙拉处理
橙子（去皮）（切片） 进行榨汁处理


In [10]:
from functools import partial

# 创建专门榨汁的函数（默认去皮切片）
make_juice = partial(process_fruit, action="榨汁", peeled=True, sliced=True)

# 创建专门制作水果沙拉的函数
make_salad = partial(
    process_fruit, action="制作沙拉", peeled=True, sliced=True
)

# 创建带皮处理整果的函数
process_whole = partial(process_fruit, peeled=False, sliced=False)

# 使用简化后的函数
print(make_juice("苹果"))  # 苹果（去皮）（切片） 进行榨汁处理
print(make_juice("橙子"))  # 橙子（去皮）（切片） 进行榨汁处理
print(make_salad("香蕉"))  # 香蕉（去皮）（切片） 进行制作沙拉处理
print(process_whole("桃子", "清洗"))  # 桃子 进行清洗处理

苹果（去皮）（切片） 进行榨汁处理
橙子（去皮）（切片） 进行榨汁处理
香蕉（去皮）（切片） 进行制作沙拉处理
桃子 进行清洗处理


### 6.1.4 装饰器与装饰器工厂

装饰器是Python中元编程（Metaprogramming）的一个典型应用。元编程指的是编写能操作代码的代码，即程序能够在运行时创建、修改或分析其他程序（包括自身）。装饰器作为一种元编程技术，允许我们在不修改原函数代码的情况下增强其功能，是Python实现代码复用和抽象的重要工具。

In [15]:
# 装饰器：一个简单的元编程示例
def trace_calls(func):
    """跟踪函数调用的装饰器"""

    def wrapper(*args, **kwargs):
        print(f"进入函数: {func.__name__}")
        result = func(*args, **kwargs)
        print(f"离开函数: {func.__name__}")
        return result

    return wrapper


# 使用装饰器
@trace_calls
def prepare_fruit_salad(fruits: list[str]):
    """准备水果沙拉"""
    return f"用{', '.join(fruits)}制作的水果沙拉"


print(prepare_fruit_salad(["苹果", "香蕉", "橙子"]))
# 输出:
# 进入函数: prepare_fruit_salad
# 离开函数: prepare_fruit_salad
# 用苹果, 香蕉, 橙子制作的水果沙拉

进入函数: prepare_fruit_salad
离开函数: prepare_fruit_salad
用苹果, 香蕉, 橙子制作的水果沙拉


In [12]:
def log_factory(level="INFO"):
    """日志装饰器工厂：根据级别记录日志"""

    def decorator(func):
        """装饰器：接收被装饰函数，返回包装函数"""

        def wrapper(*args, **kwargs):
            """包装函数：实际执行增强逻辑"""
            print(f"[{level}] 开始执行: {func.__name__}")
            result = func(*args, **kwargs)
            print(f"[{level}] 执行完成: {func.__name__}")
            return result

        return wrapper  # 装饰器返回包装函数

    return decorator  # 装饰器工厂返回装饰器


# 使用装饰器工厂
@log_factory(level="DEBUG")
def process_fruit(fruit: str):
    """处理水果的函数"""
    return f"处理{fruit}完成"


# 调用函数
print(process_fruit("苹果"))
# 输出:
# [DEBUG] 开始执行: process_fruit
# [DEBUG] 执行完成: process_fruit
# 处理苹果完成

[DEBUG] 开始执行: process_fruit
[DEBUG] 执行完成: process_fruit
处理苹果完成


In [13]:
def validation_factory(valid_fruits=None):
    """验证装饰器工厂：确保处理的是有效水果"""
    if valid_fruits is None:
        valid_fruits = ["苹果", "香蕉", "橙子", "葡萄"]

    def decorator(func):
        def wrapper(fruit, action):
            if fruit not in valid_fruits:
                raise ValueError(
                    f"不支持的水果类型: {fruit}，有效类型: {valid_fruits}"
                )
            return func(fruit, action)

        return wrapper

    return decorator


# 创建只允许特定水果的装饰器
@validation_factory(valid_fruits=["苹果", "橙子"])
def juice_only(fruit, action):
    return f"{fruit}被用来{action}"


# 正常调用
print(juice_only("苹果", "榨汁"))  # 正常执行

# 异常调用
try:
    print(juice_only("香蕉", "榨汁"))  # 抛出异常
except ValueError as e:
    print(f"验证失败: {e}")

苹果被用来榨汁
验证失败: 不支持的水果类型: 香蕉，有效类型: ['苹果', '橙子']


## 6.2 常见设计模式

### 6.2.1 单例模式

In [1]:
class SingletonByNew:
    """基于 __new__ 方法的单例模式"""

    _instance = None  # 类变量，用于存储唯一实例

    def __new__(cls, *args, **kwargs):
        # 如果实例不存在，则创建新实例
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            print("创建单例实例")
        return cls._instance

    def __init__(self):
        # 防止 __init__ 被多次调用
        if not hasattr(self, "_initialized"):
            self.data = {}  # 初始化数据
            self._initialized = True
            print("初始化单例实例")


# 测试
s1 = SingletonByNew()  # 输出: 创建单例实例\n初始化单例实例
s2 = SingletonByNew()  # 不会再次创建和初始化

s1.data["host"] = "localhost"
print(f"s1.data: {s1.data}")  # 输出: {'host': 'localhost'}
print(f"s2.data: {s2.data}")  # 输出: {'host': 'localhost'}
print(f"是同一个实例: {s1 is s2}")  # 输出: True

创建单例实例
初始化单例实例
s1.data: {'host': 'localhost'}
s2.data: {'host': 'localhost'}
是同一个实例: True


In [2]:
def singleton(cls):
    """单例装饰器"""
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
            print(f"创建 {cls.__name__} 实例")
        return instances[cls]

    return get_instance


@singleton
class ConfigManager:
    """配置管理器"""

    def __init__(self):
        self.settings = {"host": "localhost", "port": 8080, "debug": False}
        print("初始化配置管理器")


@singleton
class Logger:
    """日志记录器"""

    def __init__(self):
        self.log_level = "INFO"
        print("初始化日志记录器")


# 测试装饰器实现的单例
config1 = ConfigManager()  # 输出: 创建 ConfigManager 实例\n初始化配置管理器
config2 = ConfigManager()  # 不会再次创建和初始化
print(f"配置管理器是同一个实例: {config1 is config2}")  # 输出: True

logger1 = Logger()  # 输出: 创建 Logger 实例\n初始化日志记录器
logger2 = Logger()  # 不会再次创建和初始化
print(f"日志记录器是同一个实例: {logger1 is logger2}")  # 输出: True

初始化配置管理器
创建 ConfigManager 实例
配置管理器是同一个实例: True
初始化日志记录器
创建 Logger 实例
日志记录器是同一个实例: True


In [ ]:
class SingletonMeta(type):
    """单例元类"""

    _instances = {}

    def __call__(cls, *args, **kwargs):
        # 如果该类还没有实例，则创建新实例
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
            print(f"创建 {cls.__name__} 实例（通过元类）")
        return cls._instances[cls]


class DatabaseConnection(metaclass=SingletonMeta):
    """数据库连接（使用元类实现单例）"""

    def __init__(self):
        self.connection_string = "localhost:1234/mydb"
        print("初始化数据库连接")


# 测试元类实现的单例
db1 = DatabaseConnection()
# 输出: 初始化数据库连接\n创建 DatabaseConnection 实例（通过元类）

db2 = DatabaseConnection()  # 不会再次创建和初始化

print(f"是同一个数据库连接: {db1 is db2}")  # 输出: True

初始化数据库连接
创建 DatabaseConnection 实例（通过元类）
是同一个数据库连接: True


### 6.2.2 工厂模式

In [ ]:
# 定义水果接口
class Fruit:
    def get_description(self):
        pass

    def get_price(self):
        pass


# 具体水果类
class Apple(Fruit):
    def get_description(self):
        return "新鲜的红苹果"

    def get_price(self):
        return 3.0


class Orange(Fruit):
    def get_description(self):
        return "多汁的橙子"

    def get_price(self):
        return 4.0


# 水果工厂
class FruitFactory:
    """水果工厂，负责创建水果对象"""

    @staticmethod
    def create_fruit(fruit_type):
        if fruit_type == "apple":
            return Apple()
        elif fruit_type == "orange":
            return Orange()
        else:
            raise ValueError(f"不支持的水果类型: {fruit_type}")


# 使用工厂模式
def create_fruit_basket(fruit_list):
    """创建水果篮"""
    basket = []
    total_price = 0

    for fruit_type in fruit_list:
        fruit = FruitFactory.create_fruit(fruit_type)
        basket.append(fruit.get_description())
        total_price += fruit.get_price()

    return basket, total_price


# 测试工厂模式
basket, price = create_fruit_basket(["apple", "orange"])
print(f"水果篮: {basket}")
print(f"总价: {price}元")  # 输出: 7.0

水果篮: ['新鲜的红苹果', '多汁的橙子']
总价: 7.0元
